In [ ]:
import os
import json
from PIL import Image
import pandas as pd

알약 별, 기본 데이터에서의 등장 횟수를 불러온다.

In [ ]:
pill_count={}
with open('./pill_count.txt','r') as f:
    for l in f:
        a,b=map(int,l.strip().split('\t'))
        pill_count[a]=b

In [ ]:
pill_code=pill_count.keys()
len(pill_code)

In [ ]:
pill_images={}
for code in pill_code:
    pill_images[code]=[]
pill_images

In [ ]:
pill_count

In [ ]:
os.getcwd()

각 알약 별로, annotation에 따라서 자른 단일 이미지를 만든다.</br>
추후, 분류기를 만드는 데에 사용할 수 있다.</br>
혹은, annotation 검토 용으로도 사용 가능하다.</br>
간혹 다른 알약과 위치가 바뀐 경우가 있는데, 이런 것을 찾을 때에도 유용하다.</br>
결과물을 참조하여 annotation 위치를 수작업으로 갱신하거나, 그냥 해당 데이터를 지우는 방법을 사용할 수 있다.

In [ ]:
images=sorted(os.listdir('./additional_training_data/train_cleaned'))
annot_root='./additional_training_data/train_annotations'
annots_t=sorted(os.listdir(annot_root)) # 알약 조합 목록
annots_list=[]
images_list=[]

for image in images:
    images_list.append(image[:-4])

# 각 알약 조합에 대해
for annot_set in annots_t:
    # 약 목록을 구한다.
    pill_list=sorted(os.listdir(os.path.join(annot_root,annot_set)))

    # 각 약에 대해
    for pill in pill_list:
        # 관심 대상이 아니면 넘어간다.
        if int(pill[2:])-1 not in pill_images.keys():
            continue
        # 각 약이 어떤 이미지에 대한 annotation을 갖는지에 대한 list
        imgs=sorted(os.listdir(os.path.join(annot_root,annot_set,pill)))
        # 갖고 있는 이미지에 대해
        for img in imgs:
            # 이미지가 존재한다면 잘라서 저장한다.
            if img[:-5] in images_list:
                with open(str(os.path.join(annot_root,annot_set,pill,img)), "r", encoding="utf-8") as f:
                    pill_data = json.load(f)
                    x,y,w,h=pill_data['annotations'][0]['bbox']
                    image_file=Image.open(os.path.join('./additional_training_data/train_cleaned',img[:-5]+'.png')).convert("RGBA")
                    x1, y1 = max(0, int(x)), max(0, int(y))
                    x2, y2 = min(image_file.width, int(x+w)), min(image_file.height, int(y+h))
                    if x1>x2:
                        continue
                    elif y1>y2:
                        continue
                    image_crop=image_file.crop((x1, y1, x2, y2))

                    image_crop.save(f"./pill_image/{str(int(pill[2:])-1)}/{img[:-5]}.png", format="PNG")

In [ ]:
sorted(pill_code)

이후, 각 이미지를 확인하여 잘못된 레이블이 있으면 아래 코드를 활용하여 수정하였습니다.</br>
잘못된 이미지와 매핑된 경우 어떤 이미지와 원래 매핑되었는지 알 수 없으므로 annotation 파일과 잘린 이미지를 삭제하였습니다.

In [1047]:
with open(str(os.path.join('./additional_training_data/train_annotations','K-005391-038162-041768_json','K-041768','K-005391-038162-041768_0_2_0_2_70_000_200.json')), "r", encoding="utf-8") as f:
    pill_data = json.load(f)
    x,y,w,h=pill_data['annotations'][0]['bbox']
    image_file=Image.open(os.path.join('./additional_training_data/train_cleaned','K-005391-038162-041768_0_2_0_2_70_000_200.png')).convert("RGBA")
    x1, y1 = max(0, int(x)), max(0, int(y))
    x2, y2 = min(image_file.width, int(x+w)), min(image_file.height, int(y+h))
    image_crop=image_file.crop((x1, y1, x2, y2))

    image_crop.save(f"./pill_image/41767/K-005391-038162-041768_0_2_0_2_70_000_200.png", format="PNG")

In [1001]:
with open(str(os.path.join('./additional_training_data/train_annotations','K-001900-003544-006563-016548_json','K-003544','K-001900-003544-006563-016548_0_2_0_2_90_000_200.json')), "r", encoding="utf-8") as f:
    pill_data = json.load(f)
    x,y,w,h=pill_data['annotations'][0]['bbox']
    image_file=Image.open(os.path.join('./additional_training_data/train_cleaned','K-001900-003544-006563-016548_0_2_0_2_90_000_200'+'.png')).convert("RGBA")
    x1, y1 = max(0, int(x)), max(0, int(y))
    x2, y2 = min(image_file.width, int(x+w)), min(image_file.height, int(y+h))
    image_crop=image_file.crop((x1, y1, x2, y2))

    image_crop.save(f"./pill_image/3543/K-001900-003544-006563-016548_0_2_0_2_90_000_200.png", format="PNG")

In [ ]:
str(int(pill[2:])-1),img[:-5]

각 알약 조합 별, 모든 알약에 대해 정보가 존재하는 이미지들을 구한다.

In [ ]:
images=sorted(os.listdir('./additional_training_data/train_cleaned'))
annot_root='./additional_training_data/train_annotations'
annots_t=sorted(os.listdir(annot_root)) # 알약 조합 목록
annots_list=[]
images_list=[]
# 각 알약 조합에 대해
for annot_set in annots_t:
    # 약 목록을 구한다.
    pill_list=sorted(os.listdir(os.path.join(annot_root,annot_set)))
    # 첫 약이 가지는 json 파일 목록 (각 이미지에 대한 json 파일 목록)
    base_images=sorted(os.listdir(os.path.join(annot_root,annot_set,pill_list[0])))
    b=len(base_images)

    min_count=500
    for pill in pill_list:
        # 만약 데이터가 필요 없는 약이라면 생략한다.
        if int(pill[2:])-1 not in pill_count.keys():
            continue

    # 각 약에 대해
    for pill in pill_list:
        if int(pill[2:])-1 in [3350,3482]:
            print('ho')
        if int(pill[2:])-1 not in pill_images.keys():
            continue
        if len(pill_images[int(pill[2:])-1])==20:
            continue
        # 각 약이 어떤 이미지에 대한 annotation을 갖는지에 대한 list
        imgs=sorted(os.listdir(os.path.join(annot_root,annot_set,pill)))
        # 모든 약의 annotation이 없는 이미지여도 추가
        for img in imgs:
            if img not in base_images:
                base_images.append(img)
        # 모든 약의 annotation이 없는 이미지여도 추가
        for img in imgs:
            if img not in base_images:
                base_images.append(img)
            if img not in pill_images[int(pill[2:])-1]:
                pill_images[int(pill[2:])-1].append(img)
            # pill_count[int(pill[2:])-1]+=1
    
    
    # annotation이 모두 존재하는 이미지 중 첫 번째만 추가
    if len(base_images)>0:
        annots_list.append(base_images[0][:-5])

for img in images:
    images_list.append(img[:-4])
len(annots_list), annots_list[0], len(images_list),images_list[0]

In [ ]:
for k,v in pill_images.items():
    print(k,len(v))

전체 파일 목록을 추출

In [ ]:
images_set=set(images_list)
annots_set=set(annots_list)
# 이미지에 대해, 모든 알약의 annotation이 있는 경우의 이름들을 모음
available_files=sorted(list(images_set.intersection(annots_set)))
len(available_files)

In [ ]:
os.getcwd()

In [ ]:
pill_count

In [ ]:
with open('./annots_without_image.txt','w') as f:
    for annot in annots_list:
        if annot not in images_list:
            f.write(str(annot)+'\n')

In [ ]:
with open('./images_without_annot.txt','w') as f:
    for image in images_list:
        if image not in annots_list:
            f.write(str(image)+'\n')

In [ ]:
for f in available_files:
    print(f)
    break

이미지 단위로 annotation 파일들을 결합합니다.

In [ ]:
#json 파일 통합

base_path = "./additional_training_data"
ann_root = os.path.join(base_path, "train_annotations")

output_path = os.path.join(base_path, "train_annots_v2")
os.makedirs(output_path, exist_ok=True)

image_id_offset = 1499
annotation_id_offset = 5691
category_set = set()

# 알약 조합 별 폴더 명
for root in sorted(os.listdir(ann_root)):

    # 현재 조합의 알약 종류
    pills=sorted(os.listdir(os.path.join(ann_root,root)))

    # 현재 조합의 이미지 목록
    images_list=[]
    for pill in pills:
        images_list_p=os.listdir(os.path.join(ann_root,root,pill))
        for i in images_list_p:
            if i not in images_list:
                images_list.append(i)

    if len(images_list)==0:
        continue

    # 각 이미지 별 JSON 파일 생성
    for img in images_list:

        images=[]
        annotations=[]
        categories=[]

        # 대응하는 이미지가 없는 annotation이면 continue
        if img[:-5] not in available_files:
            continue
        # 각 알약에 대해서
        for pill in pills:
            # 만약 이번 알약은 해당 이미지에 대한 annotation이 없다면 넘어갑니다.
            if img not in os.listdir(os.path.join(ann_root,root,pill)):
                continue
            p_id=int(pill[2:])
            # 해당 이미지의 json 파일을 엽니다.
            with open(str(os.path.join(ann_root,root,pill,img)), "r", encoding="utf-8") as f:
                pill_data = json.load(f)
                cur_image=pill_data['images']
                cur_image[0]['id']+=image_id_offset
                an=pill_data['annotations']

                # 만약 annotation의 bbox에 대해 4개의 좌표가 주어지지 않았다면
                # 해당 파일은 삭제하고 건너뜁니다.
                if len(an[0]['bbox'])!=4:
                    os.system(f'rm -rf "{os.path.join(ann_root,root,pill,img)}"')
                    continue

                # 만약 pill_code에 없는 annotation이라면 넘어갑니다.
                if p_id-1 not in pill_code:
                    continue

                # 이외의 경우 데이터를 추가합니다.
                an[0]['category_id']=(p_id-1)
                an[0]['image_id']=image_id_offset
                an[0]['id']+=annotation_id_offset

                annotation_id_offset+=1
                annotations.extend(an)

                if len(images)==0:
                    images.extend(pill_data["images"])
                    image_id_offset+=1

                cat=pill_data["categories"]
                cat[0]['id']=(p_id-1)
                cat[0]['name']=pill_data["images"][0]['dl_name']
                categories.extend(cat)
        output_coco={
            "images": images,
            "annotations": annotations,
            "categories": categories
        }
        with open(os.path.join(output_path,img), "w", encoding="utf-8") as f:
            json.dump(output_coco, f, ensure_ascii=False, indent=4)

이제 모든 이미지의 json 파일을 결합합니다.

In [ ]:
output_all_coco = {
    "images": [],
    "annotations": [],
    "categories": []
}
files = os.listdir(os.path.join(base_path, "train_annots_v2"))

for file in files:
    with open(os.path.join(base_path, "train_annots_v2", file), "r", encoding="utf-8") as f:
        data=json.load(f)
        for image in data['images']:
            output_all_coco['images'].append(image)
        for annot in data['annotations']:
            output_all_coco['annotations'].append(annot)
        for cat in data['categories']:
            output_all_coco['categories'].append(cat)

t=pd.DataFrame(output_all_coco['categories'])
t=t.drop_duplicates()
output_all_coco['categories']=t.to_dict(orient='records')

save_path = os.path.join(base_path, "train_v2.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(output_all_coco, f, ensure_ascii=False, indent=4)

print("train_v2.json 생성됨:", save_path)

In [ ]:
len(output_all_coco['categories'])

In [ ]:
len(output_all_coco['images'])